In [4]:
from __future__ import unicode_literals, print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import pandas as pd

import os
import re
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
SOS_token = 0
EOS_token = 0
MAX_LENGTH = 20

class Lang: # 딕셔너리를 만들기 위한 클래스
    def __init__(self): # 단어의 인덱스를 저장하기 위한 컨테이너를 초기화
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"} # SOS(Start of Sequence): 문장의 시작, EOS(End of Sequence): 문장의 끝
        self.n_words = 2 # SOS와 EOS에 대한 카운트

    def addSentence(self, sentence): # 문장을 단어 단위로 분리한 후 컨테이너(word)에 추가
        for word in sentence.split(' '):
            self.addWord(word)
    
    def addWord(self, word): # 컨테이너에 단어가 없다면 추가되고, 있다면 카운트를 업데이트
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else: 
            self.word2count[word] += 1

In [6]:
def normalizeString(df, lang):
    sentence = df[lang].str.lower() # 소문자로 변환
    sentence = sentence.str.replace('[A-Za-z\s]+' '') # a-z, A-Z, ..., ?, ! 등을 제외하고 모두 공백으로 변환
    sentence = sentence.str.normalize('NFD') # 유니코드 정규화 방식
    sentence = sentence.str.encode('ascii', errors = 'ignore').str.decode('utf-8') # Unicode를 ASCII로 변환
    return sentence

def read_sentence(df, lang1, lang2):
    sentence1 = normalizeString(df, lang1) # 데이터셋의 첫 번째 열(영어)
    sentence2 = normalizeString(df, lang2) # 데이터셋의 두 번째 열(프랑스어)
    return sentence1, sentence2

def read_file(loc, lang1, lang2):
    df = pd.read_csv(loc, delimiter = '\t', header = None, names = [lang1, lang2])
    return df

def process_data(lang1, lang2):
    df = read_file('D:/kim/kim/DP/data/%s-%s.txt'%(lang1, lang2), lang1, lang2) # 데이터셋 불러오기
    sentence1, sentence2 = read_sentence(df, lang1, lang2)

    input_lang = Lang()
    output_lang = Lang()
    pairs = []
    for i in range(len(df)):
        if len(sentence1[i].split(' ')) < MAX_LENGTH and len(sentence2[i].split(' ')) < MAX_LENGTH:
            full = [sentence1[i], sentence2[i]] # 첫 번째와 두 번째 열을 합쳐서 저장
            input_lang.addSentence(sentence1[i]) # 입력으로 영어를 사용
            output_lang.addSentence(sentence2[i]) # 출력으로 프랑스어를 사용
            pairs.append(full) # pairs에는 입력과 출력이 합쳐진 것을 사용

    return input_lang, output_lang, pairs

In [7]:
def indexesFromSentence(lang, sentence): # 문장을 단어로 분리하고 인덱스를 반환
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence): # 딕셔너리에서 단어에 대한 인덱스를 가져오고 문장 끝에 토큰을 추가
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype = torch.long, device = device).view(-1, 1)

def tensorsFromPair(input_lang, output_lang, pair): # 입력과 출력 문장을 텐서로 변환하여 반환
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [8]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, embbed_dim, num_layers):
        super(Encoder, self).__init__()
        self.input_dim = input_dim # 인코더에서 사용할 입력층
        self.embbed_dim = embbed_dim # 인코더에서 사용할 임베딩 계층
        self.hidden_dim = hidden_dim # 인코더에서 사용할 은닉층(이전 은닉층)
        self.num_layers = num_layers # 인코더에서 사용할 GRU의 계층 개수
        self.embedding = nn.Embedding(input_dim, self.embbed_dim) # 임베딩 계층 초기화
        self.gru = nn.GRU(self.embbed_dim, self.hidden_dim, num_layers = self.num_layers) # 임베딩 차원, 은닉층 차원, GRU의 계층 개수를 이용하여 GRU 게층을 초기화

    def forward(self, src):
        embedded = self.embedding(src).view(1, 1, -1) # 임베딩 처리
        outputs, hidden = self.gru(embedded) # 임베딩 결과를 GRU 모델에 적용
        return outputs, hidden

In [10]:
class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, embbed_dim, num_layers):
        super(Decoder, self).__init__()

        self.embbed_dim = embbed_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        self.embedding = nn.Embedding(output_dim, self.embbed_dim) # 임베딩 계층 초기화
        self.gru = nn.GRU(self.embbed_dim, self.hidden_dim, num_layers = self.num_layers) # GRU 계층 초기화
        self.out = nn.Linear(self.hidden_dim, output_dim) # 선형 계층 초기화
        self.softmax = nn.LogSoftmax(dim = 1)

    def forward(self, input, hidden):
        input = input.view(1, -1) # 입력을 (1, 배치 크기)로 변경
        embedded = F.relu(self.embedding(input))
        output, hidden = self.gru(embedded, hidden)
        prediction = self.softmax(self.out(output[0]))
        return prediction, hidden